In [53]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Menlo; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")

In [114]:
import numpy as np
from operator import and_
from operator import or_
from itertools import product, combinations, chain
from functools import reduce

inv = lambda a, bitlen: (((1<<bitlen)-1)^a)
funcs = [   lambda a, b, n:       a  &  b , # AND
            lambda a, b, n:       a  |  b , #  OR
            lambda a, b, n:       a  ^  b , # XOR
            lambda a, b, n:      (a  |  b), # NOR
            lambda a, b, n: inv(a,n) &  b , # IMP
            lambda a, b, n: inv(b,n) &  a , # IMP
        ]
def powerset(iterable, minlen = 1):
    """Yields all possible subsets of the iterable.

        >>> list(powerset([1, 2, 3]))
        [(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

    :func:`powerset` will operate on iterables that aren't :class:`set`
    instances, so repeated elements in the input will produce repeated elements
    in the output. Use :func:`unique_everseen` on the input to avoid generating
    duplicates:

        >>> seq = [1, 1, 0]
        >>> list(powerset(seq))
        [(), (1,), (1,), (0,), (1, 1), (1, 0), (1, 0), (1, 1, 0)]
        >>> from more_itertools import unique_everseen
        >>> list(powerset(unique_everseen(seq)))
        [(), (1,), (0,), (1, 0)]

    """
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(minlen, len(s) + 1))
    
def bin_codes(nvars, with_neg = True):
    bitlen = 1 << nvars
    codes = []
    for i in range(nvars):
        base = (1 << (1 << i)) - 1
        deg = np.arange(0, bitlen, 1 << (i + 1), dtype=int)
        num = np.sum(base * (1 << deg))
        # print(f"{num:0{bitlen}b} -> {num}")
        codes.append(num)
        if with_neg:
            codes.append(inv(num, bitlen))
    return codes
nbits = 3
bitlen = 1 << nbits
codes = bin_codes(nbits)
    
inputs = set()
for items in powerset(codes, minlen = 2):
    num = reduce(or_, items)
    inputs.add(num)
    
outputs = set()
for a, b in combinations(inputs, 2):
    for func in funcs:
        out = func(a, b, bitlen)
        outputs.add(out)

print(f'Impossible: {len(set(range(1 << bitlen)).difference(sorted(outputs)))}')
# for num in set(range(1 << bitlen)).difference(sorted(outputs)):
#     print(f"{num:0{bitlen}b} -> {num}")
    
inputs2 = inputs | outputs

outputs2 = set()
for a, b in combinations(inputs2, 2):
    for func in funcs:
        out = func(a, b, bitlen)
        outputs2.add(out)
        
# print('Possible:')
# for num in sorted(outputs2):
#     print(f"{num:0{bitlen}b} -> {num}")


print(f'Impossible: {len(set(range(1 << bitlen)).difference(sorted(outputs2)))}')
# for num in set(range(1 << bitlen)).difference(sorted(outputs2)):
#     print(f"{num:0{bitlen}b} -> {num}")
        

Impossible: 50
Impossible: 0


In [133]:
import numpy as np
from operator import and_
from operator import or_
from itertools import product, combinations, chain
from functools import reduce

inv = lambda a, bitlen: (((1<<bitlen)-1)^a)
funcs = [   lambda a, b, n:       a  &  b , # AND
            lambda a, b, n:       a  |  b , #  OR
            lambda a, b, n:       a  ^  b , # XOR
            lambda a, b, n:      (a  |  b), # NOR
            lambda a, b, n: inv(a,n) &  b , # IMP
            lambda a, b, n: inv(b,n) &  a , # IMP
        ]
def pset(iterable, minlen = 1, maxlen = None):
    """Yields all powersets of the iterable of size between minlen and maxlen.  """
    s = list(iterable)
    if maxlen is None:
        maxlen = len(s)
    return chain.from_iterable(combinations(s, r) for r in range(minlen, maxlen + 1))

def bin_codes(nvars, with_neg = True):
    ''' Generate binary codes for {nvars} variables.
    By default, also produce the negated codes.
    Examples: 
    >> codes = bin_codes(2, with_neg = True ) -> [0b0101, 0b1010, 0b0011, 0b1100]
    >> codes = bin_codes(2, with_neg = False) -> [0b01010101, 0b00110011, 0b00001111]
    '''
    bitlen = 1 << nvars
    codes = []
    for i in range(nvars):
        base = (1 << (1 << i)) - 1
        deg = np.arange(0, bitlen, 1 << (i + 1), dtype=int)
        num = np.sum(base * (1 << deg))
        # print(f"{num:0{bitlen}b} -> {num}")
        codes.append(num)
        if with_neg:
            codes.append(inv(num, bitlen))
    return codes

def or_up_to_depth(codes, maxdepth):
    '''Returns all possible truth tables obtainable from given codes using an OR tree with maximum depth of {maxdepth}'''
    
    for i in range(1, maxdepth+1):
        new_codes = np.unique(np.bitwise_or.outer(codes, codes))
        print(len(new_codes))
        if len(new_codes) == len(codes):
            # print(f'Aborted at depth {i}')
            break
        else:
            codes = new_codes
    return codes
        

nbits = 4
bitlen = 1 << nbits
mask = (1 << bitlen) - 1
codes = bin_codes(nbits)

print(sorted(codes))
depth = 5
outputs = set()
for i in range(1, depth + 1):
    inputs = or_up_to_depth(codes, maxdepth = 5)
    ORed  = np.bitwise_or.outer(inputs, inputs)
    ANDed = np.bitwise_and.outer(inputs, inputs)
    IMPed = np.bitwise_and.outer(inputs, np.bitwise_xor(inputs, mask))
    NORed = np.bitwise_xor(ORed, mask)
    XORed = np.bitwise_xor.outer(inputs, inputs)

    outputs.update(n for q in chain(ORed, ANDed, IMPed, NORed, XORed,) for n in q.ravel())

    remaining = mask+1 - len(outputs)
    print(f'Remaining: {remaining}')
    if not remaining:
        print(f'All possible functions are achievable with at most {i} cycles')
        break
    # for num in set(range(1 << bitlen)).difference(sorted(outputs2)):
    #     print(f"{num:0{bitlen}b} -> {num}")
    codes = list(set().union(codes, outputs))


[255, 3855, 13107, 21845, 43690, 52428, 61680, 65280]
33
81
81
[  255  3855  4095 13107 13311 16191 16383 21845 22015 24415 24575 30583
 30719 32639 32767 43690 43775 44975 45055 48059 48127 49087 49151 52428
 52479 53199 53247 56797 56831 57311 57343 61166 61183 61423 61439 61680
 61695 62451 62463 62965 62975 63479 63487 64250 64255 64507 64511 64764
 64767 65021 65023 65278 65279 65280 65295 65331 65343 65365 65375 65399
 65407 65450 65455 65467 65471 65484 65487 65501 65503 65518 65519 65520
 65523 65525 65527 65530 65531 65532 65533 65534 65535]
33
81
81
Remaining: 61770
60974


In [62]:
codes = bin_codes(nbits)
for num in combinations(codes):
    print(f"{num:0{bitlen}b}")


01010101
00110011
00001111
10101010
11001100
11110000


In [56]:
def eval_tree_3(bits, operators):
    '''Evaluates the tree of height 3'''
    op1, op2, op3 = operators
    a,b,c,d = bits
    return op3(op1(a,b), op2(c,d))

for operators in product(*[(_and, _or),(_and, _or),(_and, _or, _xor)]):
    result = eval_tree_3(codes, operators)
    print(f"{result:0{bitlen}b} -> {result}")

0000000000000001 -> 1
0001000100011111 -> 4383
0001000100011110 -> 4382
0000000100010001 -> 273
0001111111111111 -> 8191
0001111011101110 -> 7918
0000000000000111 -> 7
0111011101111111 -> 30591
0111011101111000 -> 30584
0000011101110111 -> 1911
0111111111111111 -> 32767
0111100010001000 -> 30856
